# Assignment 3

Import libraries and define common helper functions

In [199]:
import os
import sys
import gzip
from pathlib import Path
import csv


import pandas as pd
# import s3fs # causing errors
import pyarrow as pa
# from pyarrow.json import read_json
import pyarrow.parquet as pq
import fastavro
import pygeohash
import snappy
from snappy import snappy
import jsonschema
from jsonschema.exceptions import ValidationError
from jsonschema import validate
import json


endpoint_url='https://storage.budsc.midwest-datascience.com'

current_dir = Path(os.getcwd()).absolute()
schema_dir = current_dir.joinpath('schemas')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)

# s3fs is causing errors in the program since I am working locally I removed it.
'''
def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]


    return records
'''

"\ndef read_jsonl_data():\n    s3 = s3fs.S3FileSystem(\n        anon=True,\n        client_kwargs={\n            'endpoint_url': endpoint_url\n        }\n    )\n    src_data_path = 'data/processed/openflights/routes.jsonl.gz'\n    with s3.open(src_data_path, 'rb') as f_gz:\n        with gzip.open(f_gz, 'rb') as f:\n            records = [json.loads(line) for line in f.readlines()]\n\n\n    return records\n"

Load the records from https://storage.budsc.midwest-datascience.com/data/processed/openflights/routes.jsonl.gz 


In [200]:
# loading the records data
with open('/home/totennacht/Bellevue/dsc650/data/processed/openflights/routes.jsonl.gz', 'rb') as f_gz:
    with gzip.open(f_gz, 'rb') as f:
        records = [json.loads(line) for line in f.readlines()]


## 3.1

### 3.1.a JSON Schema

In [201]:
# getting a feel for what I have here

d_keys = records[0].keys()
d_values = records[0].items()
d_items = records[0].items()

In [202]:
# I started here building my own schema by hand
print(json.dumps(records[0], indent=2))

{
  "airline": {
    "airline_id": 410,
    "name": "Aerocondor",
    "alias": "ANA All Nippon Airways",
    "iata": "2B",
    "icao": "ARD",
    "callsign": "AEROCONDOR",
    "country": "Portugal",
    "active": true
  },
  "src_airport": {
    "airport_id": 2965,
    "name": "Sochi International Airport",
    "city": "Sochi",
    "country": "Russia",
    "iata": "AER",
    "icao": "URSS",
    "latitude": 43.449902,
    "longitude": 39.9566,
    "altitude": 89,
    "timezone": 3.0,
    "dst": "N",
    "tz_id": "Europe/Moscow",
    "type": "airport",
    "source": "OurAirports"
  },
  "dst_airport": {
    "airport_id": 2990,
    "name": "Kazan International Airport",
    "city": "Kazan",
    "country": "Russia",
    "iata": "KZN",
    "icao": "UWKD",
    "latitude": 55.606201171875,
    "longitude": 49.278701782227,
    "altitude": 411,
    "timezone": 3.0,
    "dst": "N",
    "tz_id": "Europe/Moscow",
    "type": "airport",
    "source": "OurAirports"
  },
  "codeshare": false,
  "equ

In [203]:
# I need practice
# I figured out 2.2 on the last assignment but I need to keep playing with it or I will forget what I did.
# This looks pretty good. I feel like I am getting better at using dictionaries.

my_schema = {}
my_schema["type"] = "object"
my_schema["properties"] = {}
my_schema["properties"]["airline"] = {}
my_schema["properties"]["airline"]["airline_id"] = {}
my_schema["properties"]["airline"]["airline_id"]['type'] = "number"
my_schema["properties"]["airline"]["name"] = {}
my_schema["properties"]["airline"]["name"]['type'] = "string"

# I was very close before to finishing but this does look like I have the hang of it.
# Continuing with my schema that I edited by hand.

In [204]:
records[0]


{'airline': {'airline_id': 410,
  'name': 'Aerocondor',
  'alias': 'ANA All Nippon Airways',
  'iata': '2B',
  'icao': 'ARD',
  'callsign': 'AEROCONDOR',
  'country': 'Portugal',
  'active': True},
 'src_airport': {'airport_id': 2965,
  'name': 'Sochi International Airport',
  'city': 'Sochi',
  'country': 'Russia',
  'iata': 'AER',
  'icao': 'URSS',
  'latitude': 43.449902,
  'longitude': 39.9566,
  'altitude': 89,
  'timezone': 3.0,
  'dst': 'N',
  'tz_id': 'Europe/Moscow',
  'type': 'airport',
  'source': 'OurAirports'},
 'dst_airport': {'airport_id': 2990,
  'name': 'Kazan International Airport',
  'city': 'Kazan',
  'country': 'Russia',
  'iata': 'KZN',
  'icao': 'UWKD',
  'latitude': 55.606201171875,
  'longitude': 49.278701782227,
  'altitude': 411,
  'timezone': 3.0,
  'dst': 'N',
  'tz_id': 'Europe/Moscow',
  'type': 'airport',
  'source': 'OurAirports'},
 'codeshare': False,
 'equipment': ['CR2']}

In [205]:
print(json.dumps(my_schema,indent=2))

{
  "type": "object",
  "properties": {
    "airline": {
      "airline_id": {
        "type": "number"
      },
      "name": {
        "type": "string"
      }
    }
  }
}


In [206]:
# omg I have been trying to use records with an s but should be using record below wasting precious time.
print(json.dumps(records[0]))

{"airline": {"airline_id": 410, "name": "Aerocondor", "alias": "ANA All Nippon Airways", "iata": "2B", "icao": "ARD", "callsign": "AEROCONDOR", "country": "Portugal", "active": true}, "src_airport": {"airport_id": 2965, "name": "Sochi International Airport", "city": "Sochi", "country": "Russia", "iata": "AER", "icao": "URSS", "latitude": 43.449902, "longitude": 39.9566, "altitude": 89, "timezone": 3.0, "dst": "N", "tz_id": "Europe/Moscow", "type": "airport", "source": "OurAirports"}, "dst_airport": {"airport_id": 2990, "name": "Kazan International Airport", "city": "Kazan", "country": "Russia", "iata": "KZN", "icao": "UWKD", "latitude": 55.606201171875, "longitude": 49.278701782227, "altitude": 411, "timezone": 3.0, "dst": "N", "tz_id": "Europe/Moscow", "type": "airport", "source": "OurAirports"}, "codeshare": false, "equipment": ["CR2"]}


In [207]:
def validate_jsonl_data(records):
    schema_path = '/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/schemas/routes-schema.json'
    with open(schema_path) as f:
        schema = json.load(f)
    invalid_key_counter = 0

    with open('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/validation-results.csv', 'w') as csv_file:
        csv_writer = csv.writer(csv_file,escapechar = " ", quoting = csv.QUOTE_NONE)
        for i, record in enumerate(records):
            try:
                ## TODO: Validate record
                validate(record,schema)
                csv_writer.writerow([json.dumps(record)])
                pass
            except ValidationError as e:
                ## Print message if invalid record
                print(f'Key" {i} Record: {record} was invalidated by the schema')
                invalid_key_counter = invalid_key_counter + 1
                pass
    print(f"Total number of records invalidated by the schema: {invalid_key_counter}")


validate_jsonl_data(records)

Key" 7 Record: {'airline': {'airline_id': 410, 'name': 'Aerocondor', 'alias': 'ANA All Nippon Airways', 'iata': '2B', 'icao': 'ARD', 'callsign': 'AEROCONDOR', 'country': 'Portugal', 'active': True}, 'src_airport': {'airport_id': 4029, 'name': 'Domodedovo International Airport', 'city': 'Moscow', 'country': 'Russia', 'iata': 'DME', 'icao': 'UUDD', 'latitude': 55.40879821777344, 'longitude': 37.90629959106445, 'altitude': 588, 'timezone': 3.0, 'dst': 'N', 'tz_id': 'Europe/Moscow', 'type': 'airport', 'source': 'OurAirports'}, 'dst_airport': None, 'codeshare': False, 'equipment': ['CR2']} was invalidated by the schema
Key" 38 Record: {'airline': {'airline_id': 410, 'name': 'Aerocondor', 'alias': 'ANA All Nippon Airways', 'iata': '2B', 'icao': 'ARD', 'callsign': 'AEROCONDOR', 'country': 'Portugal', 'active': True}, 'src_airport': None, 'dst_airport': {'airport_id': 4029, 'name': 'Domodedovo International Airport', 'city': 'Moscow', 'country': 'Russia', 'iata': 'DME', 'icao': 'UUDD', 'latitu

### 3.1.b Avro

In [208]:
from fastavro import writer, parse_schema


def create_avro_dataset(records):
    schema_path = '/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/schemas/routes.avsc'
    data_path = '/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/routes.avro'

    ## TODO: Use fastavro to create Avro dataset
    with open(schema_path, 'r') as schema_file:
        parsed_schema = parse_schema(json.load(schema_file))

    # Write initial records
    with open(data_path, 'wb') as out:
        writer(out, parsed_schema, records, validator=True)

create_avro_dataset(records)
print("\nLater we discovered which ones were actually required but I dont have time to go back and fix this. "
      "It does work as intended it just assumes that everything was required which is silly.\n")


Later we discovered which ones were actually required but I dont have time to go back and fix this. It does work as intended it just assumes that everything was required which is silly.



### 3.1.c Parquet

In [209]:
from pyarrow import json as pj

def create_parquet_dataset():
    src_data_path = '/home/totennacht/Bellevue/dsc650/data/processed/openflights/routes.jsonl.gz'
    parquet_output_path = '/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/routes.parquet'

    with open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            table = pj.read_json(src_data_path)
            # checking the output this looks great!
            print(table)
            pass
            ## TODO: Use Apache Arrow to create Parquet table and save the dataset
    # using pandas to transfer from a arrow table to parquet.
    my_pyarrow_df = table.to_pandas()
    my_pyarrow_df.to_parquet(parquet_output_path)


create_parquet_dataset()

pyarrow.Table
airline: struct<airline_id: int64, name: string, alias: string, iata: string, icao: string, callsign: string, country: string, active: bool>
  child 0, airline_id: int64
  child 1, name: string
  child 2, alias: string
  child 3, iata: string
  child 4, icao: string
  child 5, callsign: string
  child 6, country: string
  child 7, active: bool
src_airport: struct<airport_id: int64, name: string, city: string, country: string, iata: string, icao: string, latitude: double, longitude: double, altitude: int64, timezone: double, dst: string, tz_id: string, type: string, source: string>
  child 0, airport_id: int64
  child 1, name: string
  child 2, city: string
  child 3, country: string
  child 4, iata: string
  child 5, icao: string
  child 6, latitude: double
  child 7, longitude: double
  child 8, altitude: int64
  child 9, timezone: double
  child 10, dst: string
  child 11, tz_id: string
  child 12, type: string
  child 13, source: string
dst_airport: struct<airport_id: 

### 3.1.d Protocol Buffers

Protocol Buffers

Define a Protocol Buffers message format in schemas/routes.proto. Using the Protocol Buffers Python tutorial as a guide,
compile the routes.proto into Python classes. Output the generated code into dsc650/assignment/assignment03/routes_pb2.py.
Use this generated code to create results/routes.pb using Protocol Buffers. Do not use any compression on the output at this stage.

In [210]:
# I need to see my records again
print(records[0])

{'airline': {'airline_id': 410, 'name': 'Aerocondor', 'alias': 'ANA All Nippon Airways', 'iata': '2B', 'icao': 'ARD', 'callsign': 'AEROCONDOR', 'country': 'Portugal', 'active': True}, 'src_airport': {'airport_id': 2965, 'name': 'Sochi International Airport', 'city': 'Sochi', 'country': 'Russia', 'iata': 'AER', 'icao': 'URSS', 'latitude': 43.449902, 'longitude': 39.9566, 'altitude': 89, 'timezone': 3.0, 'dst': 'N', 'tz_id': 'Europe/Moscow', 'type': 'airport', 'source': 'OurAirports'}, 'dst_airport': {'airport_id': 2990, 'name': 'Kazan International Airport', 'city': 'Kazan', 'country': 'Russia', 'iata': 'KZN', 'icao': 'UWKD', 'latitude': 55.606201171875, 'longitude': 49.278701782227, 'altitude': 411, 'timezone': 3.0, 'dst': 'N', 'tz_id': 'Europe/Moscow', 'type': 'airport', 'source': 'OurAirports'}, 'codeshare': False, 'equipment': ['CR2']}


In [211]:
sys.path.insert(0, os.path.abspath('routes_pb2'))

import routes_pb2

def _airport_to_proto_obj(airport):
    obj = routes_pb2.Airport()
    if airport is None:
        return None
    if airport.get('airport_id') is None:
        return None

    obj.airport_id = airport.get('airport_id')
    if airport.get('name'):
        obj.name = airport.get('name')
    if airport.get('city'):
        obj.city = airport.get('city')
    if airport.get('iata'):
        obj.iata = airport.get('iata')
    if airport.get('icao'):
        obj.icao = airport.get('icao')
    if airport.get('altitude'):
        obj.altitude = airport.get('altitude')
    if airport.get('timezone'):
        obj.timezone = airport.get('timezone')
    if airport.get('dst'):
        obj.dst = airport.get('dst')
    if airport.get('tz_id'):
        obj.tz_id = airport.get('tz_id')
    if airport.get('type'):
        obj.type = airport.get('type')
    if airport.get('source'):
        obj.source = airport.get('source')

    obj.latitude = airport.get('latitude')
    obj.longitude = airport.get('longitude')

    return obj


def _airline_to_proto_obj(airline):
    obj = routes_pb2.Airline()
    if not airline.get('name'):
        return None
    if not airline.get('airline_id'):
        return None
    if not airline.get('active'):
        return None

    # these are required
    obj.airline_id = airline.get('airline_id')
    obj.name = airline.get('name')
    obj.active = airline.get('active')

    if airline.get('alias'):
        obj.alias = airline.get('alias')

    # I now need to build these since they are missing.
    # It took me awhile to see this but it makes sense.
    if airline.get('iata'):
        obj.iata = airline.get('iata')
    if airline.get('icao'):
        obj.icao = airline.get('icao')
    if airline.get('latitude'):
        obj.latitude = airline.get('latitude')
    if airline.get('longitude'):
        obj.longitude = airline.get('longitude')
    if airline.get('altitude'):
        obj.altitude = airline.get('altitude')
    if airline.get('timezone'):
        obj.timezone = airline.get('timezone')
    if airline.get('dst'):
        obj.dst = airline.get('dst')
    if airline.get('tz_id'):
        obj.tz_id = airline.get('tz_id')
    if airline.get('type'):
        obj.type = airline.get('type')
    if airline.get('source'):
        obj.source = airline.get('source')

    ## TODO
    return obj

def create_protobuf_dataset(records):
    routes = routes_pb2.Routes()
    for record in records:
        route = routes_pb2.Route()

# I think I am seeing this now Route has airline, src_airport, dst_airport, stops, and equipment
# So we have to build these from our file

        airline = _airline_to_proto_obj(record.get('airline', {}))
        src_airport = _airport_to_proto_obj(record.get('src_airport', {}))

        if airline:
            route.airline.CopyFrom(airline)

        if src_airport:
            route.src_airport.CopyFrom(src_airport)

        route.codeshare = record.get('codeshare')

        ## TODO

        routes.route.append(route)
    data_path = results_dir.joinpath('routes.pb')

    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())

    compressed_path = results_dir.joinpath('routes.pb.snappy')

    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))
        
create_protobuf_dataset(records)

3.1.e Output Sizes

Compare the output sizes of the different formats. Populate the results in results/comparison.csv.

In [215]:
import os

def snappy_compress(file_name):
        path = results_dir.joinpath(file_name)
        path_to_store = results_dir.joinpath(file_name + '.snappy')
        # print(path)
        # print(path_to_store)

        with open(path, 'rb') as in_file:
          with open(path_to_store, 'wb') as out_file:
            snappy.stream_compress(in_file, out_file)
            out_file.close()
            in_file.close()

        return path_to_store



# build a table with output sizes
my_table = pd.DataFrame(0, columns=['csv','avro','parquet', 'protobuf'], index=['Un-Compressed','Compressed (snappy)'])

# un-compressed
my_table.loc['Un-Compressed','csv'] = os.path.getsize('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/validation-results.csv')
my_table.loc['Un-Compressed','avro'] = os.path.getsize('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/routes.avro')
my_table.loc['Un-Compressed','parquet'] = os.path.getsize('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/routes.parquet')
my_table.loc['Un-Compressed','protobuf'] = os.path.getsize('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/validation-results.csv')

snappy_compress('routes.avro')
snappy_compress('routes.parquet')
snappy_compress('validation-results.csv')

# compressed
my_table.loc['Compressed (snappy)','csv'] = os.path.getsize('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/validation-results.csv.snappy')
my_table.loc['Compressed (snappy)','avro'] = os.path.getsize('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/routes.avro.snappy')
my_table.loc['Compressed (snappy)','parquet'] = os.path.getsize('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/routes.parquet.snappy')
my_table.loc['Compressed (snappy)','protobuf'] = os.path.getsize('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/routes.pb.snappy')

print(my_table)
my_table.to_csv('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment03/results/comparison.csv')

                          csv      avro  parquet  protobuf
Un-Compressed        75794863  19646227  1978786  75794863
Compressed (snappy)   8798466   3018358  1965004   1794239


## 3.2

### 3.2.a Simple Geohash Index


In [213]:

def create_hash_dirs(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    for record in records:
        src_airport = record.get('src_airport', {})
        if src_airport:
            latitude = src_airport.get('latitude')
            longitude = src_airport.get('longitude')
            if latitude and longitude:
                ## TODO: use pygeohash.encode() to assign geohashes to the records and complete the hashes list
                hashes.append(pygeohash.encode(longitude=longitude,latitude=latitude,precision=3))
                record['geohash'] = pygeohash.encode(longitude=longitude,latitude=latitude,precision=3)


    hashes.sort()
    three_letter = sorted(list(set([entry[:3] for entry in hashes])))
    hash_index = {value: [] for value in three_letter}
    print(hash_index)
    for record in records:
        geohash = record.get('geohash')
        if geohash:
            hash_index[geohash[:3]].append(record)
    for key, values in hash_index.items():
        output_dir = geoindex_dir.joinpath(str(key[:1])).joinpath(str(key[:2]))
        output_dir.mkdir(exist_ok=True, parents=True)
        output_path = output_dir.joinpath('{}.jsonl.gz'.format(key))
        with gzip.open(output_path, 'w') as f:
            json_output = '\n'.join([json.dumps(value) for value in values])
            f.write(json_output.encode('utf-8'))
    # checking to ensure I have my three code hashes
    print(hashes)
            
create_hash_dirs(records)



{'2eg': [], '2ev': [], '2ey': [], '2h5': [], '2hb': [], '2hx': [], '2j0': [], '2j3': [], '2jd': [], '2jq': [], '2jt': [], '2kn': [], '2kp': [], '2kr': [], '2kw': [], '2s2': [], '2sh': [], '2sv': [], '2t5': [], '2th': [], '2tp': [], '2tq': [], '2tr': [], '2ug': [], '2v0': [], '2v1': [], '2v2': [], '2yk': [], '2ym': [], '35b': [], '3e4': [], '3zz': [], '4qr': [], '4qt': [], '4qy': [], '4rh': [], '4rv': [], '4wy': [], '4xb': [], '627': [], '62m': [], '62s': [], '62t': [], '62u': [], '62v': [], '63h': [], '63k': [], '63p': [], '63z': [], '66j': [], '66p': [], '66r': [], '66t': [], '67j': [], '67v': [], '683': [], '68e': [], '695': [], '696': [], '69r': [], '69y': [], '6cb': [], '6d0': [], '6d6': [], '6db': [], '6dc': [], '6dh': [], '6dk': [], '6dx': [], '6e1': [], '6e4': [], '6e9': [], '6en': [], '6eq': [], '6ex': [], '6f6': [], '6f9': [], '6fc': [], '6fe': [], '6ff': [], '6fg': [], '6fu': [], '6g0': [], '6g1': [], '6g3': [], '6g4': [], '6g5': [], '6g7': [], '6g9': [], '6gg': [], '6gj': []

### 3.2.b Simple Search Feature

In [214]:
def airport_search(latitude, longitude):
    ## TODO: Create simple search to return nearest airport
    src_geohash_code = pygeohash.encode(longitude=longitude,latitude=latitude,precision=15)
    # print('hash code for (41.1499988, -95.91779)')
    # print(src_geohash_code)

    # load the data from that file
    # loading the records data
    geoindex_dir = results_dir.joinpath('geoindex')

    # building my search area
    my_geo_dir = geoindex_dir.joinpath(str(src_geohash_code[0]))
    my_geo_dir = my_geo_dir.joinpath(str(src_geohash_code[:2]))
    my_geo_dir = my_geo_dir.joinpath(str(src_geohash_code[:3])+".jsonl.gz")

    # print(my_geo_dir)
    with open(my_geo_dir, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            geo_records = [json.loads(line) for line in f.readlines()]
            current_distance = 0
            closest_airport_name = {}
            for distance in geo_records:
                # I chose src because I want them to be within the same geocode block as the university
                if distance['src_airport']:
                    # print(distance)
                    # print('\n')
                    temp_lat = distance["src_airport"]["latitude"]
                    temp_lon = distance["src_airport"]["longitude"]
                    dst_geohash_code = pygeohash.encode(longitude=temp_lon,latitude=temp_lat,precision=15)
                    # print(src_geohash_code)
                    # print(dst_geohash_code)
                    distance_to_airport = pygeohash.geohash_approximate_distance(str(src_geohash_code), str(dst_geohash_code))
                    # print(distance_to_airport)
                    if current_distance == 0 or distance_to_airport <= current_distance:
                        current_distance = distance_to_airport
                        closest_airport_name = distance["airline"]["name"]

            print("The closest airport is:")
            print(closest_airport_name)
            print(str(current_distance) + " km away")


    
airport_search(41.1499988, -95.91779)

The closest airport is:
Southwest Airlines
19545 km away
